In [1]:
## Importing standard libraries

%pylab inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.python.keras.utils.np_utils import to_categorical


Populating the interactive namespace from numpy and matplotlib


In [2]:
## Set figure size to 20x10

from pylab import rcParams
rcParams['figure.figsize'] = 10,10

In [3]:
## Read data from the CSV file

data = pd.read_csv('./input/train.csv')
parent_data = data.copy()    ## Always a good idea to keep a copy of original data
ID = data.pop('id')

In [4]:
data.shape

(990, 193)

In [5]:
## Since the labels are textual, so we encode them categorically

y = data.pop('species')
y = LabelEncoder().fit(y).transform(y)
print(y.shape)

(990,)


In [6]:
## Most of the learning algorithms are prone to feature scaling
## Standardising the data to give zero mean =)

X = StandardScaler().fit(data).transform(data)
print(X.shape)

(990, 192)


In [7]:
## We will be working with categorical crossentropy function
## It is required to further convert the labels into "one-hot" representation

y_cat = to_categorical(y)
print(y_cat.shape)

(990, 99)


In [8]:
## Developing a layered model for Neural Networks
## Input dimensions should be equal to the number of features
## We used softmax layer to predict a uniform probabilistic distribution of outcomes

model = Sequential()
model.add(Dense(1024,input_dim=192,kernel_initializer = 'uniform', activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(99, activation='softmax'))

In [9]:
## Error is measured as categorical crossentropy or multiclass logloss
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics = ["accuracy"])

In [10]:
## Fitting the model on the whole training data
history = model.fit(X,y_cat,batch_size=32,
                    epochs=400,verbose=0)

In [16]:
# validation:

# scores = pd.DataFrame(history.history)
# min(scores['val_loss'])
# scores.loc[20:,["loss", "val_loss"]].plot()

KeyError: 'val_loss'

In [13]:
test = pd.read_csv('./input/test.csv')
index = test.pop('id')
test = StandardScaler().fit(test).transform(test)
#yPred = model.predict_proba(test)
yPred = model.predict(test)
# print(model.score)

AttributeError: 'Sequential' object has no attribute 'score'

In [14]:
## Converting the test predictions in a dataframe as depicted by sample submission

output = pd.DataFrame(yPred,index=index,columns=sort(parent_data.species.unique()))

In [ ]:
fp = open('submission_nn_kernel_output.csv','w')
fp.write(output.to_csv())